## Train a model with bike rental data using XGBoost algorithm
### Training log1p(count) dataset
###  Model is trained with XGBoost installed in notebook instance
###  In the later examples, we will train using SageMaker's XGBoost algorithm

In [ ]:
# Install xgboost in notebook instance.
#### Command to install xgboost
!conda install -y -c conda-forge xgboost

In [ ]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

In [ ]:
column_list_file = 'bike_train_column_list.txt'
train_file = 'bike_train.csv'
validation_file = 'bike_validation.csv'
test_file = 'bike_test.csv'

In [ ]:
columns = ''
with open(column_list_file,'r') as f:
    columns = f.read().split(',')

In [ ]:
columns

In [ ]:
# Specify the column names as the file does not have column header
df_train = pd.read_csv(train_file,names=columns)
df_validation = pd.read_csv(validation_file,names=columns)

In [ ]:
df_train.head()

In [ ]:
df_validation.head()

In [ ]:
X_train = df_train.iloc[:,1:] # Features: 1st column onwards 
y_train = df_train.iloc[:,0].ravel() # Target: 0th column

X_validation = df_validation.iloc[:,1:]
y_validation = df_validation.iloc[:,0].ravel()

In [ ]:
# XGBoost Training Parameter Reference: 
#   https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
regressor = xgb.XGBRegressor(max_depth=5,eta=0.1,subsample=0.7,num_round=150)

In [ ]:
regressor

In [ ]:
regressor.fit(X_train,y_train, eval_set = [(X_train, y_train), (X_validation, y_validation)])

In [ ]:
df_train['count'].describe()

In [ ]:
eval_result = regressor.evals_result()

In [ ]:
training_rounds = range(len(eval_result['validation_0']['rmse']))

In [ ]:
print(training_rounds)

In [ ]:
plt.scatter(x=training_rounds,y=eval_result['validation_0']['rmse'],label='Training Error')
plt.scatter(x=training_rounds,y=eval_result['validation_1']['rmse'],label='Validation Error')
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('RMSE')
plt.title('Training Vs Validation Error')
plt.legend()

In [ ]:
xgb.plot_importance(regressor)

In [ ]:
df = pd.read_csv('bike_all.csv')

In [ ]:
df.head()

In [ ]:
X_test = df.iloc[:,1:]
print(X_test[:5])

In [ ]:
result = regressor.predict(X_test)

In [ ]:
result[:5]

In [ ]:
df.head()

In [ ]:
df['count_predicted'] = result

In [ ]:
df.head()

In [ ]:
# Negative Values are predicted
df['count_predicted'].describe()

In [ ]:
df[df['count_predicted'] < 0]

In [ ]:
def adjust_count(x):
    if x < 0:
        return 0
    else:
        return x

In [ ]:
df['count_predicted'] = df['count_predicted'].map(adjust_count)

In [ ]:
df[df['count_predicted'] < 0]

In [ ]:
plt.boxplot([df['count'],df['count_predicted']], labels=['actual','predicted'])
plt.title('Box Plot - Actual, Predicted')
plt.ylabel('Target')
plt.grid(True)

In [ ]:
# Over prediction and Under Prediction needs to be balanced
# Training Data Residuals
residuals = (df['count_predicted'] - df['count'])

plt.hist(residuals)
plt.grid(True)
plt.xlabel('(Predicted - Actual)')
plt.ylabel('Count')
plt.title('Residuals Distribution')
plt.axvline(color='g')


In [ ]:
import sklearn.metrics as metrics
print("RMSE: {0}".format(metrics.mean_squared_error(df['count'].map(np.expm1),df['count_predicted'].map(np.expm1))**.5))

In [ ]:
# Metric Use By Kaggle
def compute_rmsle(y_true, y_pred):
    if type(y_true) != np.ndarray:
        y_true = np.array(y_true)
        
    if type(y_pred) != np.ndarray:
        y_pred = np.array(y_pred)
     
    return(np.average((np.log1p(y_pred) - np.log1p(y_true))**2)**.5)

In [ ]:
print("RMSLE: {0}".format(compute_rmsle(df['count'].map(np.expm1),df['count_predicted'].map(np.expm1))))

In [ ]:
# Prepare Data for Submission to Kaggle
df_test = pd.read_csv(test_file,parse_dates=['datetime'])

In [ ]:
df_test.head()

In [ ]:
X_test =  df_test.iloc[:,1:] # Exclude datetime for prediction

In [ ]:
X_test.head()

In [ ]:
result = regressor.predict(X_test)

In [ ]:
result[:5]

In [ ]:
np.expm1(result)

In [ ]:
# Convert result to actual count
df_test["count"] = np.expm1(result)

In [ ]:
df_test.head()

In [ ]:
df_test[df_test["count"] < 0]

In [ ]:
df_test[['datetime','count']].to_csv('predicted_count.csv',index=False)

In [ ]:
# RMSLE (Kaggle) Scores
# Test 1: 1.32
# Test 2 (added new feature): 0.61646
# Test 3 0.4